# Lab 10: MPI

MPI is a standard specification of a message passing interface

<img align="right"  src="images/mpi1.png" alt="Drawing" style="width: 300px;"/>

<div style="text-align: left"> 

    SPMD (Single program multiple data):
    
    - Execution starts in parallel
    
    - MPI implements SPMD
    
    -Static parallelism: #processes doesn't change
    
    
    Fork / Join :
    
    - Execution starts serial
    
    - New processes created at fork
    
    - Used in pthreads
    
    - Dynamic parallelism
    
    
</div>


### MPI - system components

<img  src="images/mpi2.png" alt="Drawing" style="width: 600px;"/>

- Node: A single host on network

- Rank: Process executing the MPI program

### MPI - programmer view

<img  src="images/mpi3.png" alt="Drawing" style="width: 600px;"/>

- Nodes are transparent to the programmer, only ranks matter

- Communicator: Group of ranks that can communicate

- Comm world: Communicator that includes all the ranks

^ Images source: HPML lectures

### Point-to-Point Communication

- send() : Comm.send(self, obj, int dest, int tag=0)


    - obj: object to be sent
    - dest: Rank of destination process
    - Tag: Used to differentiate among messages
    
    
- recv() : Comm.recv(self, buf=None, int source=ANY_SOURCE, int tag=ANY_TAG, Status status=None)

    
    - buf: Optional buffer for the data to be received
    - source: Rank of source process
    - Tag: Used to differentiate among messages
    - status: information about the data received, e.g. rank of source, tag

- comm.Get_rank(): Returns the rank of current process

- comm.Get_size(): Returns the total number of processes

#### Python objects (pickle under the hood)

Use lowercase send() and recv() for python objects

In [1]:
%%writefile ex1.py

from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = {'a': 7, 'b': 3.14}
    comm.send(data, dest=1, tag=11)
elif rank == 1:
    data = comm.recv(source=0, tag=11)
    
    print('Message {} received at rank{}:'.format(data, rank))


Writing ex1.py


In [2]:
!mpiexec -n 2 python ex1.py

Message {'a': 7, 'b': 3.14} received at rank1:
[a-10-27-13-185.dynapool.vpn.nyu.edu:08060] PMIX ERROR: ERROR in file gds_ds12_lock_pthread.c at line 206
[a-10-27-13-185.dynapool.vpn.nyu.edu:08058] PMIX ERROR: ERROR in file gds_ds12_lock_pthread.c at line 206


#### Numpy arrays

Use uppercase Send() and Recv()

Buffer argument must be specified as [data, TYPE (MPI.DOUBLE)]

In [3]:
%%writefile ex2.py

from mpi4py import MPI
import numpy

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

# passing MPI datatypes explicitly
if rank == 0:
    data = numpy.arange(1000, dtype='i')
    comm.Send([data, MPI.INT], dest=1, tag=77)
elif rank == 1:
    data = numpy.empty(1000, dtype='i')
    comm.Recv([data, MPI.INT], source=0, tag=77)
    
    print('Message {} received at rank{}:'.format(data, rank))

# automatic MPI datatype discovery
if rank == 0:
    data = numpy.arange(100, dtype=numpy.float64)
    comm.Send(data, dest=1, tag=13)
elif rank == 1:
    data = numpy.empty(100, dtype=numpy.float64)
    comm.Recv(data, source=0, tag=13)
    
    print('Message {} received at rank{}:'.format(data, rank))

Writing ex2.py


In [4]:
!mpiexec -n 2 python ex2.py

Message [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 24

In [5]:
%%writefile ex3.py

from mpi4py import MPI

comm = MPI.COMM_WORLD

my_rank = comm.Get_rank()
p = comm.Get_size()

if my_rank != 0:
    message = 'Hello from the other rank {}'.format(my_rank)
    comm.send(message, dest = 0)

else:
    for pid in range(1,p):
        message = comm.recv(source = pid)
        print('Process {} receives message: {}'.format(my_rank, message))
#rank 0 receives 3 form the other 3 ranks

Writing ex3.py


In [6]:
!mpiexec -n 4 python ex3.py

--------------------------------------------------------------------------
There are not enough slots available in the system to satisfy the 4
slots that were requested by the application:

  python

Either request fewer slots for your application, or make more slots
available for use.

A "slot" is the Open MPI term for an allocatable unit where we can
launch a process.  The number of slots available are defined by the
environment in which Open MPI processes are run:

  1. Hostfile, via "slots=N" clauses (N defaults to number of
     processor cores if not provided)
  2. The --host command line parameter, via a ":N" suffix on the
     hostname (N defaults to 1 if not provided)
  3. Resource manager (e.g., SLURM, PBS/Torque, LSF, etc.)
  4. If none of a hostfile, the --host command line parameter, or an
     RM is present, Open MPI defaults to the number of processor cores

In all the above cases, if you want Open MPI to default to the number
of hardware threads instead of the number of

### Non-blocking communication:

Isend(), Irecv() are non blocking:
    
    Process can continue execution and wait later

In [7]:
%%writefile ex4.py

from mpi4py import MPI
import time

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = {'a': 7, 'b': 3.14}
    req = comm.isend(data, dest=1, tag=11)
    req.wait()
    print('Process {} sent {}'.format(rank, data))
    
elif rank == 1:
    req = comm.irecv(source=0, tag=11)
    # do something before waiting the message to arrive (large) and w could do things first
    time.sleep(2)
    #sleep 2 seconds, then we receive the request from rank 0
    data = req.wait()
    print('Process {} received {}'.format(rank, data))


Writing ex4.py


In [8]:
!mpiexec -n 2 python ex4.py

Process 0 sent {'a': 7, 'b': 3.14}
Process 1 received {'a': 7, 'b': 3.14}
[a-10-27-13-185.dynapool.vpn.nyu.edu:08068] PMIX ERROR: ERROR in file gds_ds12_lock_pthread.c at line 206
[a-10-27-13-185.dynapool.vpn.nyu.edu:08066] PMIX ERROR: ERROR in file gds_ds12_lock_pthread.c at line 206


### Performance gain

In [9]:
%%writefile ex5.py

from mpi4py import MPI
import timeit

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

N = 10000000

def parSum():
    if rank == 0: 
        s = sum(range(N//2))
        comm.send(s,dest=2)
        
    elif rank == 1:
        s = sum(range(N//2+1,N))
        comm.send(s,dest=2)
        
    elif rank == 2:
        s1 = comm.recv(source=0)
        s2 = comm.recv(source=1)
        print (s1+s2)


def serSum():
    s = sum(range(N))

if rank == 0:
    
    tp = timeit.Timer("parSum()","from __main__ import parSum")
    print ('Parallel time: {:.4f} sec'.format(tp.timeit(number=10))) 

    ts = timeit.Timer("serSum()","from __main__ import serSum")
    print ('Serial time: {:.4f} sec'.format(ts.timeit(number=10))) 



Writing ex5.py


In [10]:
!mpiexec -n 3 python ex5.py

--------------------------------------------------------------------------
There are not enough slots available in the system to satisfy the 3
slots that were requested by the application:

  python

Either request fewer slots for your application, or make more slots
available for use.

A "slot" is the Open MPI term for an allocatable unit where we can
launch a process.  The number of slots available are defined by the
environment in which Open MPI processes are run:

  1. Hostfile, via "slots=N" clauses (N defaults to number of
     processor cores if not provided)
  2. The --host command line parameter, via a ":N" suffix on the
     hostname (N defaults to 1 if not provided)
  3. Resource manager (e.g., SLURM, PBS/Torque, LSF, etc.)
  4. If none of a hostfile, the --host command line parameter, or an
     RM is present, Open MPI defaults to the number of processor cores

In all the above cases, if you want Open MPI to default to the number
of hardware threads instead of the number of